# Snowflake

> Deploy Nixtla's TimeGPT directly in your Snowflake environment. This guide walks you through setting up TimeGPT as native Snowflake stored procedures and UDTFs, enabling you to run forecasts, detect anomalies, and evaluate models without moving your data outside Snowflake.

## Overview

The Snowflake deployment provides:

- **Native SQL Interface**: Call TimeGPT using familiar SQL stored procedures
- **Data Stays in Snowflake**: Your time series data never leaves your Snowflake environment
- **Scalable Processing**: Leverage Snowflake's distributed computing for large-scale forecasting
- **Full Feature Support**: Forecasting, anomaly detection, model evaluation, and feature contributions (SHAP)

### What Gets Deployed

| Component | Description |
|-----------|-------------|
| `NIXTLA_FORECAST` | Generate forecasts with optional confidence intervals and exogenous variables |
| `NIXTLA_DETECT_ANOMALIES` | Detect anomalies in your time series data |
| `NIXTLA_EVALUATE` | Evaluate forecast accuracy with metrics (MAPE, MAE, MSE) |
| `NIXTLA_EXPLAIN` | Get feature contributions (SHAP values) for explainable forecasts |
| `NIXTLA_FINETUNE` | Fine-tune TimeGPT on your specific data |

## Prerequisites

Before you begin, ensure you have:

1. **Snowflake Account** with permissions to:
   - Create databases, schemas, and stages
   - Create network rules and external access integrations
   - Create secrets, UDTFs, and stored procedures

2. **Nixtla API Key**: Get yours at [dashboard.nixtla.io](https://dashboard.nixtla.io)

3. **Python Environment** with the `nixtla` package installed:

```bash
pip install nixtla
```

4. **Snowflake Connection Configuration**: Create or update `~/.snowflake/config.toml`:

```toml
[default]
account = "your-account-identifier"
user = "your-username"
password = "your-password"  # Or use other auth methods
warehouse = "your-warehouse"
database = "your-database"  # Optional, can be set during deployment
schema = "your-schema"      # Optional, can be set during deployment
```

For SSO or MFA authentication, refer to [Snowflake's connection configuration documentation](https://docs.snowflake.com/en/developer-guide/python-connector/python-connector-connect).

## Installation

Run the deployment script from your terminal:

```bash
python -m nixtla.scripts.snowflake_install_nixtla
```

The script will interactively guide you through:

1. **Database Selection**: Choose or create a database for Nixtla components
2. **Schema Selection**: Choose or create a schema within the database
3. **Stage Creation**: Set up a stage to store the Nixtla package
4. **API Configuration**: Select your API endpoint and provide your API key
5. **Component Deployment**: Choose which components to deploy

### Command Line Options

You can also provide options directly:

```bash
python -m nixtla.scripts.snowflake_install_nixtla \
    --connection_name default \
    --database NIXTLA_DB \
    --schema NIXTLA_SCHEMA \
    --stage_path NIXTLA_STAGE \
    --integration_name nixtla_access_integration \
    --base_url https://api.nixtla.io
```

### API Endpoints

| Endpoint | Description |
|----------|-------------|
| `https://api.nixtla.io` | Default TimeGPT endpoint |
| `https://tsmp.nixtla.io` | TimeGPT-2 with support for all models |

## Usage Examples

Once deployed, you can use TimeGPT directly from SQL. The examples below assume deployment to `NIXTLA_DB.NIXTLA_SCHEMA`.

### Data Format

Your input data should have at minimum:

| Column | Type | Description |
|--------|------|-------------|
| `unique_id` | VARCHAR | Identifier for each time series |
| `ds` | TIMESTAMP | Timestamp of the observation |
| `y` | DOUBLE | The target value to forecast |

### Basic Forecasting

Generate a 14-day forecast with 80% and 95% confidence intervals:

```sql
CALL NIXTLA_DB.NIXTLA_SCHEMA.NIXTLA_FORECAST(
    INPUT_DATA => 'NIXTLA_DB.NIXTLA_SCHEMA.YOUR_TABLE',
    PARAMS => OBJECT_CONSTRUCT(
        'h', 14,
        'freq', 'D',
        'level', ARRAY_CONSTRUCT(80, 95)
    )
);
```

**Parameters:**
- `h`: Forecast horizon (number of periods to forecast)
- `freq`: Data frequency ('D' for daily, 'H' for hourly, 'W' for weekly, etc.)
- `level`: Confidence interval levels (optional)

**Output:**

| Column | Description |
|--------|-------------|
| `UNIQUE_ID` | Series identifier |
| `DS` | Forecast timestamp |
| `FORECAST` | Predicted value |
| `CONFIDENCE_INTERVALS` | JSON object with confidence bounds by level |

### Anomaly Detection

Detect anomalies in your time series:

```sql
CALL NIXTLA_DB.NIXTLA_SCHEMA.NIXTLA_DETECT_ANOMALIES(
    INPUT_DATA => 'NIXTLA_DB.NIXTLA_SCHEMA.YOUR_TABLE',
    PARAMS => OBJECT_CONSTRUCT(
        'level', 95,
        'freq', 'D'
    )
);
```

**Output:**

| Column | Description |
|--------|-------------|
| `UNIQUE_ID` | Series identifier |
| `DS` | Timestamp |
| `Y` | Actual value |
| `TIMEGPT` | Expected value |
| `ANOMALY` | 'True' or 'False' |
| `TIMEGPT_LO` | Lower confidence bound |
| `TIMEGPT_HI` | Upper confidence bound |

### Model Evaluation

Evaluate forecast accuracy using standard metrics:

```sql
CALL NIXTLA_DB.NIXTLA_SCHEMA.NIXTLA_EVALUATE(
    INPUT_DATA => 'NIXTLA_DB.NIXTLA_SCHEMA.YOUR_PREDICTIONS_TABLE',
    METRICS => ARRAY_CONSTRUCT('MAPE', 'MAE', 'MSE')
);
```

Your input table should include:
- `unique_id`, `ds`, `y` (actual values)
- One or more forecast columns (e.g., `TimeGPT`, `my_model`)

**Output:**

| Column | Description |
|--------|-------------|
| `UNIQUE_ID` | Series identifier |
| `FORECASTER` | Name of the forecast column evaluated |
| `METRIC` | Metric name (MAPE, MAE, MSE) |
| `VALUE` | Metric value |

### Feature Contributions (SHAP)

Understand which features drive your forecasts:

```sql
CALL NIXTLA_DB.NIXTLA_SCHEMA.NIXTLA_EXPLAIN(
    INPUT_DATA => 'NIXTLA_DB.NIXTLA_SCHEMA.YOUR_TABLE',
    PARAMS => OBJECT_CONSTRUCT(
        'h', 14,
        'freq', 'D'
    )
);
```

**Note:** Feature contributions require exogenous variables to be specified.

**Output:**

| Column | Description |
|--------|-------------|
| `UNIQUE_ID` | Series identifier |
| `DS` | Forecast timestamp |
| `FORECAST` | Predicted value |
| `FEATURE` | Feature name |
| `CONTRIBUTION` | Feature's contribution to the forecast |

### Fine-tuning

Fine-tune TimeGPT on your specific data for improved accuracy:

```sql
CALL NIXTLA_DB.NIXTLA_SCHEMA.NIXTLA_FINETUNE(
    INPUT_DATA => 'NIXTLA_DB.NIXTLA_SCHEMA.YOUR_TRAINING_TABLE',
    PARAMS => OBJECT_CONSTRUCT(
        'finetune_steps', 100
    ),
    MAX_SERIES => 1000
);
```

**Parameters:**
- `finetune_steps`: Number of fine-tuning steps (default: number of unique series)
- `MAX_SERIES`: Maximum number of series to use for fine-tuning

The procedure returns a model identifier that can be used in subsequent forecast calls.

## Troubleshooting

### Common Issues

**"External access integration not found"**

Ensure the security integration was created during deployment. Re-run the deployment script and select "Yes" for generating the security script.

**"Network rule violation" or API connection errors**

Verify that:
1. The network rule allows egress to the correct API host (`api.nixtla.io` or `tsmp.nixtla.io`)
2. The external access integration is enabled
3. Your Snowflake account allows external network access

**"Secret not found"**

The API key secret may not have been created. Re-run the deployment script and regenerate the security integration.

**MFA Authentication Required**

If your Snowflake account requires MFA, the deployment script will prompt for your authenticator code. Enter the current code from your authenticator app.

### Verifying Deployment

Check that all components are deployed:

```sql
-- List stored procedures
SHOW PROCEDURES LIKE 'NIXTLA%' IN SCHEMA NIXTLA_DB.NIXTLA_SCHEMA;

-- List UDTFs
SHOW USER FUNCTIONS LIKE 'NIXTLA%' IN SCHEMA NIXTLA_DB.NIXTLA_SCHEMA;

-- Check external access integration
SHOW EXTERNAL ACCESS INTEGRATIONS LIKE 'nixtla%';

-- Verify secrets exist
SHOW SECRETS IN SCHEMA NIXTLA_DB.NIXTLA_SCHEMA;
```

## Sample Datasets

The deployment script can create example datasets to help you get started. If you selected this option during deployment, you'll have:

| Table | Description |
|-------|-------------|
| `EXAMPLE_TRAIN` | 3 time series with 330 days of history + 14 future exogenous rows |
| `EXAMPLE_ALL_DATA` | 3 time series with 365 days including forecast column (for evaluation) |
| `EXAMPLE_ANOMALY_DATA` | 3 sensor series with 180 days and injected anomalies |

Use these to test each procedure before running on your own data.